In [15]:
import numba
from numba import jit, int32, prange, vectorize, float64, cuda
import numpy as np
import math

In [2]:
# Function that calculates the value of Pi using monte-carlo method
# Minimal use of Numpy
def native_python_monte_carlo(n):
    within_circle = 0
    
    for i in range(int(n)):
        x = np.random.random()
        y = np.random.random()
        
        if x**2 + y**2 <= 1.0:
            within_circle += 1
        
    return 4.0 * within_circle / n

In [3]:
%%time
print(native_python_monte_carlo(10000000))

3.1409408
CPU times: user 7.25 s, sys: 0 ns, total: 7.25 s
Wall time: 7.25 s


In [4]:
# Function that calculates the value of Pi using monte-carlo method
# Leveraged numpy as far as possible

def numpy_python_monte_carlo(n):
    within_circle = 0
    
    x = np.random.random(int(n))
    y = np.random.random(int(n))
        
    within_circle = np.sum( (x**2 + y**2) <= 1.0 )
        
    return 4.0 * within_circle / n

In [5]:
%%time
print(numpy_python_monte_carlo(10000000))

3.1418096
CPU times: user 132 ms, sys: 110 ms, total: 242 ms
Wall time: 241 ms


### Use @jit on both functions and see what is the speedup acheived (ignore the first run)
* Native python :
* Numpy :

* Natve python with Numba :
* Numpy wuth Numba: 

In [6]:
@jit
def native_python_monte_carlo(n):
    within_circle = 0
    
    for i in range(int(n)):
        x = np.random.random()
        y = np.random.random()
        
        if x**2 + y**2 <= 1.0:
           within_circle += 1
        
    return 4.0 * within_circle / n



<ipython-input-6-181b95bfdbf1>:2: NumbaDeprecationWarning: The 'nopython' keyword argument was not supplied to the 'numba.jit' decorator. The implicit default value for this argument is currently False, but it will be changed to True in Numba 0.59.0. See https://numba.readthedocs.io/en/stable/reference/deprecation.html#deprecation-of-object-mode-fall-back-behaviour-when-using-jit for details.
  def native_python_monte_carlo(n):


In [7]:
@jit
def numpy_python_monte_carlo(n):
    within_circle = 0
    
    x = np.random.random(int(n))
    y = np.random.random(int(n))
        
    within_circle = np.sum( (x**2 + y**2) <= 1.0 )
        
    return 4.0 * within_circle / n

<ipython-input-7-4ea981f41fd4>:2: NumbaDeprecationWarning: The 'nopython' keyword argument was not supplied to the 'numba.jit' decorator. The implicit default value for this argument is currently False, but it will be changed to True in Numba 0.59.0. See https://numba.readthedocs.io/en/stable/reference/deprecation.html#deprecation-of-object-mode-fall-back-behaviour-when-using-jit for details.
  def numpy_python_monte_carlo(n):


### What is the difference If we use eager compilation (ignore first run)?

* Native python with Numba (Lazy):
* Native python with Numba (Eager):

In [8]:


@jit(numba.float32(int32))
def native_python_monte_carlo_eager(n):
    within_circle = 0
    
    for i in range(int(n)):
        x = np.random.random()
        y = np.random.random()
        
        if x**2 + y**2 <= 1.0:
            within_circle += 1
        
    return 4.0 * within_circle / n

<ipython-input-8-8a8175256212>:1: NumbaDeprecationWarning: The 'nopython' keyword argument was not supplied to the 'numba.jit' decorator. The implicit default value for this argument is currently False, but it will be changed to True in Numba 0.59.0. See https://numba.readthedocs.io/en/stable/reference/deprecation.html#deprecation-of-object-mode-fall-back-behaviour-when-using-jit for details.
  @jit(numba.float32(int32))


In [9]:
%%time
print(native_python_monte_carlo_eager(10000000))

3.1408839225769043
CPU times: user 85.3 ms, sys: 112 µs, total: 85.4 ms
Wall time: 84.3 ms


In [10]:
%%time
@jit
def native_python_monte_carlo_lazy(n):
    within_circle = 0
    
    for i in range(int(n)):
        x = np.random.random()
        y = np.random.random()
        
        if x**2 + y**2 <= 1.0:
            within_circle += 1
        
    return 4.0 * within_circle / n

CPU times: user 342 µs, sys: 39 µs, total: 381 µs
Wall time: 347 µs


<timed exec>:2: NumbaDeprecationWarning: The 'nopython' keyword argument was not supplied to the 'numba.jit' decorator. The implicit default value for this argument is currently False, but it will be changed to True in Numba 0.59.0. See https://numba.readthedocs.io/en/stable/reference/deprecation.html#deprecation-of-object-mode-fall-back-behaviour-when-using-jit for details.


In [11]:
%%time
print(native_python_monte_carlo_lazy(10000000))

3.1410044
CPU times: user 169 ms, sys: 1.16 ms, total: 170 ms
Wall time: 168 ms


### What is the difference with and without automatic parallelization (ignore the first run)?
hint: use *numba.prange* instead of *range*
* With parallel:
* Without parallel:

In [12]:
@jit(parallel=True)
def native_python_monte_carlo_parallel(n):
    within_circle = 0
    
    for i in numba.prange(int(n)):
        x = np.random.random()
        y = np.random.random()
        
        if x**2 + y**2 <= 1.0:
            within_circle += 1
        
    return 4.0 * within_circle / n

<ipython-input-12-853d4a94477d>:1: NumbaDeprecationWarning: The 'nopython' keyword argument was not supplied to the 'numba.jit' decorator. The implicit default value for this argument is currently False, but it will be changed to True in Numba 0.59.0. See https://numba.readthedocs.io/en/stable/reference/deprecation.html#deprecation-of-object-mode-fall-back-behaviour-when-using-jit for details.
  @jit(parallel=True)


In [13]:
%%time
print(native_python_monte_carlo_parallel(10000000))

3.1413492
CPU times: user 1.36 s, sys: 39.6 ms, total: 1.4 s
Wall time: 600 ms


### Offload the computation to a GPU
* Assumptions:
    * N <= 512
* Hints:
    * Launch one block with threads <= 512

In [18]:
@cuda.jit
def native_python_monte_kernel(array_a, array_b, array_c):
    # Thread id in a 1D block
    tx = cuda.threadIdx.x
 
    if tx < an_array.size:  # Check array boundaries
        array_c[tx] = 1 if array_a[tx]**2 + array_b[tx]**2 <= 1.0  else 0

In [21]:
n = 256
array_a = np.random.random(n)
array_b = np.random.random(n)
array_c = np.zeros(n)



In [ ]:

native_python_monte_kernel[1, n](array_a, array_b, array_c)
print(4.0 * np.sum(array_c) / n)